In [ ]:
!pip install -q -U google-generativeai
!pip install openai

In [ ]:
import google.generativeai as genai
import openai
import requests
import os

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
# Create the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    system_instruction="Sos un asistente virtual que ayuda a las personas cuando se enfrentan a varios desafíos y frustraciones cuando intentan adoptar y mantener hábitos saludables. Podes combinar análisis de texto y generación de gráficos, integrados en una aplicación de seguimiento de hábitos saludables.",
)

history_chat = []
# funcion para generar la imagen desde openai
def generate_image_with_openai(prompt):
    image_response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )

    # retorna la imagen en una url
    response = image_response['data'][0]['url']
    return response

#funcion para iniciar el chat en un bucle while
def chat():
    chat_session = model.start_chat(history=history_chat)
    
    print("¡Hola! Estoy aquí para ayudarte a mantener hábitos saludables. Te brindaré análisis y gráficos personalizados para que puedas ver tu progreso y mantenerte motivado. ¡Vamos a lograrlo juntos! 🌱📊 escribe salir para terminar nuestra conversación.")
    
    while True:
        
        user_message = input("Tu: ")

        if user_message.lower() == "salir":  # Salir del chat
            break

        response = chat_session.send_message(user_message)
        print(response.text)

        # Actualizar historial con la respuesta de Gemini
        history_chat.append({'role': 'user', 'parts': [user_message]})
        history_chat.append({'role': 'model', 'parts': [response.text]})

        # Después de la primera respuesta de gemini, se pregunta si se debe generar una imagen
        if len(history_chat) == 2:  # Verifica que es la primera interacción completa osea, usuario y modelo.
            #pregunta al usuario mediante un si o no si requiere la imagen o no.
            generate_image = input("¿Deseas generar una imagen con OpenAI basada en tu mensaje? (sí/no): ").strip().lower()
            
            if generate_image == "sí" or generate_image == "si":
                image_url = generate_image_with_openai(user_message)
                print(f"Image URL: {image_url}")

                # Agregar la URL de la imagen al historial
                history_chat.append({'role': 'model', 'parts': [f"Image URL: {image_url}"]})

chat()